### Define model and cost

In [ ]:
import numpy as np
import casadi as ca


# cartpole class
class CartPole:
    def __init__(self, module='casadi'):
        if module == 'casadi':
            self.sin = ca.sin
            self.cos = ca.cos
            self.pi = ca.pi
        # else if module == 'numpy':
        #     self.sin = np.sin
        #     self.cos = np.cos
        #     self.pi = np.pi
        # else:
        #     raise TypeError

        self.mc = 2.0
        self.mp = 0.2
        self.l = 0.5
        self.gc = 9.80665

        self.lbu = -15.0
        self.ubu = 15.0
        self.lby = 1.0
        self.uby = 1.0

    def dynamics(self, x, u):
        sin = self.sin
        cos = self.cos
        pi = self.pi

        mc = self.mc
        mp = self.mp
        l = self.l
        gc = self.gc

        y = x[0]
        th = x[1]
        dy = x[2]
        dth = x[3]
        f = u[0]

        ddy = (f+mp*sin(th)*(l*dth*dth+gc*cos(th))) / (mc+mp*sin(th)*sin(th)) 
        ddth = (-f*cos(th)-mp*l*dth*dth*cos(th)*sin(th)-(mc+mp)*gc*sin(th)) / (l * (mc+mp*sin(th)*sin(th))) 
        return np.array([dy, dth, ddy, ddth])


In [ ]:
# cost calculating class
class CostFunction:
    def __init__(self, module='casadi'):
        if module == 'casadi':
            self.sin = ca.sin
            self.cos = ca.cos
            self.pi = ca.pi
        # else if module == 'numpy':
        #     self.sin = np.sin
        #     self.cos = np.cos
        #     self.pi = np.pi
        # else:
        #     raise TypeError
        pi = self.pi

        self.n_x = 4
        self.n_u = 1
        self.x_ref = np.array([0.0, pi, 0.0, 0.0])
        self.Q = np.array([2.5, 10.0, 0.01, 0.01])
        self.R = np.array([1.0])
        self.Q_f = np.array([2.5, 10.0, 0.01, 0.01])

    def stage_cost(self, x, u):
        l = 0.0
        for i in range(self.n_x):
            l += 0.5 * self.Q[i] * (x[i] - self.x_ref[i]) ** 2
        for i in range(self.n_u):
            l += 0.5 * self.R[i] * u[i]**2
        return l
    
    def terminal_cost(self, x):
        lf = 0.0
        for i in range(self.n_x):
            lf += 0.5 * self.Q_f[i] * (x[i] - self.x_ref[i]) ** 2
        return lf

    

In [ ]:
cartpole = CartPole()
cost = CostFunction()


### Approximate optimal control problem and formulate it as NLP

In [ ]:
# opti interface
opti = ca.Opti()

# dimension of state and input
n_x = 4
n_u = 1

# horizon length[s], total grids
T = 5.0
N = 200
dt = T / N

# decision variables (state, input) over the entire horizon.
xs = opti.variable(n_x, N + 1)
us = opti.variable(n_u, N)

# initial state
x0 = np.zeros(n_x)#.reshape((1, -1))

In [ ]:
# cost
J = 0.0
for i in range(N):
    J += cost.stage_cost(xs[:, i], us[:, i]) * dt
J += cost.terminal_cost(xs[:, N])

opti.minimize(J)

# state space equasion as equality constraints
opti.subject_to(xs[:, 0] == x0)
for i in range(N):
    f_array = cartpole.dynamics(xs[:, i], us[:, i])
    f = ca.vertcat(*f_array)
    x1 = xs[:, i] + f * dt
    opti.subject_to(xs[:, i + 1] == x1)

# bound for control input
for i in range(N):
    opti.subject_to(opti.bounded(cartpole.lbu, us[:, i], cartpole.ubu))
    # opti.subject_to(cartpole.lbu <= us[:, i])
    # opti.subject_to(us[:, i] <= cartpole.ubu)


In [ ]:
# not working
# cost_hist = []
# opti.callback(lambda i: cost_hist.append(opti.debug.value(J)))

### Choose solver and solve NLP

In [ ]:
# use IPOPT as NLP solver
opti.solver('ipopt', {'ipopt': {'mu_min': 0.001, 'max_iter': 1000}})

# solve
try:
    sol = opti.solve()
except RuntimeError:
    print('\n[ERROR IN SOLVE]')


### Plot trajectory

In [ ]:
import path_ocpy
from ocpy.plotter import Plotter


# xs_opt = sol.value(xs).T
# us_opt = sol.value(us).T
xs_opt = opti.debug.value(xs).T
us_opt = opti.debug.value(us).T
ts = np.array([i*dt for i in range(N + 1)])

log_dir = path_ocpy.dir + '../log/cartpole'

# print('final cost:', sol.value(J))
print('final cost:', opti.debug.value(J))

plotter = Plotter(log_dir, xs_opt, us_opt, ts)
plotter.plot(save=False)
